In [16]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

from modified_env import myEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# the base network class
from flow.networks import Network

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.controllers import IDMController, ContinuousRouter
from flow.controllers import SimLaneChangeController
from flow.core.params import SumoCarFollowingParams
from flow.controllers.routing_controllers import ConstructionRouter
from flow.core.params import SumoLaneChangeParams


# create some default parameters parameters
env_params = EnvParams()
initial_config = InitialConfig()
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=2)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)

from flow.core.params import SumoParams

sim_params = SumoParams(render=True, sim_step=1)
sim_params.color_vehicles = False


import os

fileDir='/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/constructionV3.net.xml'

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE15","gneE13","gneE35","gneE4"]
)
    
# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {"gneE15": ["gneE15","gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE35","gneE15"],
                "gneE13": ["gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE35","gneE15","gneE13"],
                "gneE35": ["gneE35","gneE15","gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE35"],
                "gneE4": ["gneE4","gneE17","gneE11","gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE36"],
               "gneE8": ["gneE8","gneE7","gneE6","gneE5","gneE8"]
               }

In [17]:
from flow.core.params import InFlows

inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE4",
           vehs_per_hour=10000,
            depart_lane="random",
            depart_speed="random",
            color="white")
# inflow.add(veh_type="human",
#            edge="gneE15",
#            vehs_per_hour=5000,
#             depart_lane="2",
#             depart_speed="random",
#             color="blue")

net_params = NetParams(
    template=fileDir,
    inflows=inflow
)

In [ ]:
flow_params = dict(
    exp_tag='test_1',
    env_name=myEnv,
    network=Network,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 10000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)

**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
